<div>
<img src="images/Übungsaufgabe.jpg" width="1500">
</div>

## Task
In this notebook an example network is created with the pandapower tool (http://www.pandapower.org/).
The network has an external grid and connected houses. For each house three types of loads are modeled:
- Photovoltaic installation in the house (pv)
- Heat pump (hp)
- Residential load (rl) 

The goal of this exercise is to understand a simple analysis of a network created with pandapower and to add a new house to it.

<div>
<img src="images/Netzbild.jpg" width="1000">
</div>

In [1]:
import pandapower as pp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

constant_dict = {"A1":40, "A2":25, "A3":65, "eff_modul1":12, "eff_modul2":8, "eff_modul3":15}

ModuleNotFoundError: No module named 'pandapower'

<div>
<img src="images/pandapower.jpg" width="1000">
</div>

## Create a sample network model

In [ ]:
def create_pp_model() -> pp.pandapowerNet:
    '''Creates a pandapower elements and connects them to model a pandapower network.'''

    # Create an empty network
    net = pp.create_empty_network()

    # Create the external grid with the corresponding bus.
    gb = pp.create_bus(net, vn_kv=0.4, name="grid_bus")
    pp.create_ext_grid(net, bus=gb, name="grid")

    # Create the house models.
    h1 = create_house_model(
        net, "house1", load=0.001, heatpump=0.0011767, pv_gen=-5.98752e-05
    )
    h2 = create_house_model(
        net, "house2", load=0.0009, heatpump=0.00121356, pv_gen=-9.02016e-05
    )
   

    # Create the lines between the houses and the grid.
    pp.create_line(net, from_bus=h1, to_bus=h2, length_km=0.1, std_type="NAYY 4x50 SE")
    pp.create_line(net, from_bus=h1, to_bus=gb, length_km=0.2, std_type="NAYY 4x50 SE")

    return net


def create_house_model(
    net: pp.pandapowerNet, name: str, load: float, pv_gen: float, heatpump: float
) -> int:
    '''Create a house model that consists of one main bus attached to three other buses with loads.

    net: The network to which the house model is added.
    name: The name of the house; also used as prefix for the named components.
    load: The household load in MW.
    pv_gen: The PV generation in MW. PV is modeled as a negative load.
    heatpump: The load of the heatpump in MW.
    return: The id of the created house bus.
    '''
    # Create the main house bus.
    main_bus = pp.create_bus(net, vn_kv=0.4, name=name)

    # Create individual buses for the different loads
    load_bus_1 = pp.create_bus(net, vn_kv=0.4, name=f"{name}_bus1")
    load_bus_2 = pp.create_bus(net, vn_kv=0.4, name=f"{name}_bus2")
    load_bus_3 = pp.create_bus(net, vn_kv=0.4, name=f"{name}_bus3")

    # Create the loads, attached to the corresponding buses.
    pp.create_load(net, bus=load_bus_1, p_mw=pv_gen, name=f"{name}_pv")
    pp.create_load(net, bus=load_bus_2, p_mw=heatpump, name=f"{name}_hp")
    pp.create_load(net, bus=load_bus_3, p_mw=load, name=f"{name}_rl")

    # Connect the load buses to the main house bus using a 50m standard cable.
    pp.create_line(
        net, from_bus=load_bus_1, to_bus=main_bus, length_km=0.05, std_type="NAYY 4x50 SE"
    )
    pp.create_line(
        net, from_bus=load_bus_2, to_bus=main_bus, length_km=0.05, std_type="NAYY 4x50 SE"
    )
    pp.create_line(
        net, from_bus=load_bus_3, to_bus=main_bus, length_km=0.05, std_type="NAYY 4x50 SE"
    )

    return main_bus




<div>
<img src="images/pv_formula.png" width="2000">
</div>

<div>
<img src="images/machine_learning.png" width="2000">
</div>

## Machine learning model for prediction

In [ ]:
def train_heatpump_model() -> LinearRegression:
    '''Trains a linear regression model for the heatpump power consumption based on temperatures.

    Returns:
        LinearRegression: The linear regression model of the heatpump consumption.
    '''
    # Read historical heat pump data from excel.
    data_ml = pd.read_excel("data_ml.xlsx")
    # Convert this data to numpy.
    data_ml = data_ml.to_numpy()
    # Define input and output of the ML model.
    X = data_ml[:, 0:2]
    Y = data_ml[:, 2:3]

    X_train = X[0:290, :]
    Y_train = Y[0:290, :]

    X_test = X[290:362, :]
    Y_test = Y[290:362, :]

    # Train Linear Regression model.
    reg = LinearRegression().fit(X_train, Y_train)
    # Make prediction on the test set to control the accuracy of the model.
    Y_pred = reg.predict(X_test)
    print(f"Test Mean Absolute Percentage Error: {mean_absolute_percentage_error(Y_test, Y_pred)}")
    
    return reg



## Calculation functions of individual load types

In [ ]:
def calc_hp(data_sheet: str, ml_model: LinearRegression, time_step: int, num_house: int) -> float:
    '''Calculation for the heat pump inside a building using a machine learning predictor
    
    data_sheet: Path to the data file.
    ml_model: Machine learning model for prediction.
    time_step: Current time step for data extraction.
    num_house: Number of the house to calculate for.
    return: The predicted power consumption of the heat pump in MW.
    '''
    # Read house data from excel.
    data_house = pd.read_excel(data_sheet + ".xlsx")
    # Extract the temperature data.
    weather_data = data_house.loc[time_step, [f"T_in{num_house}", "T_out"]]
    # Reshape the data.
    weather_data = weather_data.values.reshape(1, -1)
    # Predict power of heat pump with ml model.
    hp = ml_model.predict(weather_data)[0][0]

    return hp

def read_rl(data_sheet: str, time_step:int, num_house: int) -> float:
    '''Get data for the residential load

    data_sheet: Path to the data file.
    time_step: Current time step for data extraction.
    num_house: Number of the house to calculate for.
    return: The electrical power consumption in MW.
    '''

    # Read house data from excel.
    data_house = pd.read_excel(data_sheet + ".xlsx")
    rl = data_house.loc[time_step, f"rl{num_house}"]

    return rl    


def calc_pv(data_sheet: str, time_step: int, num_house: int) -> float:
    '''Calculation for the photovoltaic generation

    data_sheet: Path to the data file.
    time_step: Current time step for data extraction.
    num_house: Number of the house to calculate for.
    return: The generated power of the pv module in MW.
    '''

    # Read house data from excel
    data_house = pd.read_excel(data_sheet + ".xlsx")
    # Extract the irradiation data.
    irradiation = data_house.loc[time_step ,"irradiation"]
    eff_modul = constant_dict[f"eff_modul{num_house}"]/100
    A = constant_dict[f"A{num_house}"]
    eff_acdc = 0.9
    fact = 0.9
    # Calculation of the power in MW.
    pv = (A * fact * eff_modul * irradiation * eff_acdc) / 1e6
    # For generation a negative load has to be assigned.
    pv *= -1

    return pv


def assign_new_values(
    house_name: str,
    house_pv: float,
    house_hp: float,
    house_rl: float,
    net: pp.pandapowerNet
):
    '''Utility function to change the load values after creation for this example.

    house_name: Name of the house to update load values.
    house_pv: New pv value in MW:
    house_hp: New hp value in MW:
    house_rl: New rl value in MW:
    net: Pandapower network to update.
    '''

    net.load.loc[net.load["name"] == f"{house_name}_pv", "p_mw"] = house_pv
    net.load.loc[net.load["name"] == f"{house_name}_hp", "p_mw"] = house_hp
    net.load.loc[net.load["name"] == f"{house_name}_rl", "p_mw"] = house_rl



## Plot functions

In [ ]:
def plot_results(title: str, y_axis : str, plotData : list[list[float]]) -> figure:
    '''Plots multiple lists of data into one plot.
    
    title: Title of the plot.
    y_axis: Name of the y-axis label.
    plot_data: List of plot data lists to visualize.
    return: Plot of the inputs.
    '''

    p = figure(title= title, x_axis_label='Hour', y_axis_label= y_axis)
    x = list(range(24))
    p.line(x, plotData[0], legend_label="house1", color="blue")
    p.line(x, plotData[1], legend_label="house2",color="red")
    p.legend.location ="top_left"
    return p

<div>
<img src="images/dataframe.jpg" width="1000">
</div>

## Run load flow

In [ ]:
# Create network.
net = create_pp_model()

# Number of houses in the network.
NUM_HOUSES = 2

# Season for the calculation. Available seasons are spring, summer, winter.
season = "spring"

# Create machine learning model.
ml_model = train_heatpump_model()

# Initiliaze lists to store result data.
pv = [[np.NaN for _ in range(24)] for _ in range(NUM_HOUSES)]
hp = [[np.NaN for _ in range(24)] for _ in range(NUM_HOUSES)]
rl = [[np.NaN for _ in range(24)] for _ in range(NUM_HOUSES)]

# Main loop for calculation for 24 timesteps
for t in range(24): 
    for num_house in range(1, NUM_HOUSES+1):  
        # Calculate power of PVs.
        house_pv = calc_pv(f"data_{season}", t, num_house)
        # Calculate power of heat pump.
        house_hp = calc_hp(f"data_{season}", ml_model, t, num_house)
        # Read residential load (rl).
        house_rl = read_rl(f"data_{season}", t, num_house)

        assign_new_values(f"house{num_house}", house_pv, house_hp, house_rl, net)

    # Power flow calculation.
    pp.runpp(net, numba=False)

    # Store result data from the powerflow for each load.
    pv[0][t] = net.res_load.loc[0, "p_mw"] * -1
    hp[0][t] = net.res_load.loc[1, "p_mw"]
    rl[0][t] = net.res_load.loc[2, "p_mw"]

    pv[1][t] = net.res_load.loc[3, "p_mw"] * -1
    hp[1][t] = net.res_load.loc[4, "p_mw"]
    rl[1][t] = net.res_load.loc[5, "p_mw"]


# Plot the results.
pv_plot= plot_results("PV", "Generation [MW]", pv)
show(pv_plot)

hp_plot= plot_results("Heat Pump ", "Consumption [MW]", hp)
show(hp_plot)

rl_plot= plot_results("residential Load ","Consumption [MW]", rl)
show(rl_plot)


